Install PyQUBO.

まず PyQUBOをイストール

In [0]:
!pip install -q pyqubo

import PyQUBO

必要なものをインポート

In [0]:
from pyqubo import Param, Qbit, solve_qubo

Array and vector multiplying functions.

ベタだけど、行列との計算

In [0]:
def mul_arr2d(a, b):
  res = [[0,0],[0,0]]
  res[0][0] = a[0][0] * b[0][0] + a[0][1] * b[1][0]
  res[0][1] = a[0][0] * b[0][1] + a[0][1] * b[1][1]
  res[1][0] = a[1][0] * b[0][0] + a[1][1] * b[1][0]
  res[1][1] = a[1][0] * b[0][1] + a[1][1] * b[1][1]
  return res

def mul_arr2dvec(a, v):
  res = [0, 0]
  res[0] = a[0][0] * v[0] + a[0][1] * v[1]
  res[1] = a[1][0] * v[0] + a[1][1] * v[1]
  return res

def dot_vec(a, b):
  res = a[0] * b[0] + a[1] * b[1]
  return res

rewrite Mx = y to M(Av) = y. Solve array A and vector v.

Mx = y を M(Av) =  y と書き換えて、Aとv を求める感じにする。

In [0]:
# Param array M
m1 = Param('m1')
m2 = Param('m2')
m3 = Param('m3')
m4 = Param('m4')
M = [[m1, m2], [m3, m4]]

# Qbit array A、like rotation array
a1 = Qbit('a11') + 0.5*Qbit('a12') + 0.25*Qbit('a13') + 0.125*Qbit('a14')
a2 = Qbit('a21') + 0.5*Qbit('a22') + 0.25*Qbit('a23') + 0.125*Qbit('a24')

A = [[a1, a2], [-a2, a1]]

# Qbit vec
v1 = Qbit('v11') + 0.5*Qbit('v12') + 0.25*Qbit('v13') + 0.125*Qbit('v14')
v2 = Qbit('v21') + 0.5*Qbit('v22') + 0.25*Qbit('v23') + 0.125*Qbit('v24')
v = [v1, v2]

# Param vec y
y = [Param('y1'), Param('y2')]

In [0]:
MA = mul_arr2d(M, A)
MAv = mul_arr2dvec(MA, v)

MAv and y are parallel, Av and x are parallel, too.
It means maximize dot product MAv and y.

MAv と y が平行なら、Av と x も平行になるはずという理屈にする。
平行を表現するために、内積を利用し、符号を逆向きにして、できるだけ小さくするという方向へ持っていく。

In [0]:
H = -dot_vec(MAv, y)

In [0]:
model = H.compile()

In [0]:
#params = {'m1':1.0, 'm2':2.0, 'm3':3.0, 'm4':4.0, 'y1':5.0, 'y2':11.0} # x = [1, 2]
params = {'m1':-1.0, 'm2':4.0, 'm3':7.0, 'm4':2.0, 'y1':-9.0, 'y2':33.0} # x = [5, -1]

In [0]:
qubo, offset = model.to_qubo(params=params)

In [0]:
sol = solve_qubo(qubo)

In [24]:
solution, broken, energy = model.decode_solution(sol, vartype="BINARY", params=params)
print("number of broken constarint = {}".format(len(broken)))

number of broken constarint = 0


In [25]:
print(solution)

{'v23': 1, 'a21': 1, 'v21': 1, 'a23': 1, 'a11': 1, 'a13': 1, 'v24': 1, 'v12': 1, 'v13': 1, 'v11': 1, 'a22': 1, 'v22': 1, 'v14': 1, 'a12': 1, 'a24': 1, 'a14': 1}


Av will be x.

Avで欲しい x が見えるはず

In [26]:
ra1 = 1.0*solution['a11'] + 0.5*solution['a12'] + 0.25*solution['a13'] + 0.125*solution['a14']
ra2 = 1.0*solution['a21'] + 0.5*solution['a22'] + 0.25*solution['a23'] + 0.125*solution['a24']
ra = [[0.875,ra2],[-ra2,0.875]]
print(ra)

rv1 = 1.0*solution['v11'] + 0.5*solution['v12'] + 0.25*solution['v13'] + 0.125*solution['v14']
rv2 = 1.0*solution['v21'] + 0.5*solution['v22'] + 0.25*solution['v23'] + 0.125*solution['v24']
rv = [rv1, rv2]
print(rv)

av = mul_arr2dvec(ra, rv)
print(av)

[[0.875, 1.875], [-1.875, 0.875]]
[1.875, 1.875]
[5.15625, -1.875]


Real x is [5,1]. not bad.
This is a rough way, then it is OK.

まぁ、近い。他のものを試すとそれほど近くないものもあるが、やり方が粗いので、まぁ、そんなものか。

And chek MAv.

最後に Mx (=MAv) を求めてみる。

In [27]:
m = [[params['m1'],params['m2']],[params['m3'],params['m4']]]
val = mul_arr2dvec(m, av)
print(val)

[-12.65625, 32.34375]
